In this notebook, I will explore how to make the model more resilient to attacks.

**Task: Adversarial attacks on the model, implementation of
defensive mechanisms**
1. Modify the model or the training dataset to make it
more resistant to attacks (for example, by using adversarial
training, normalizing the input data, or using dropout).
2. Test the protected model on the same adversarial
examples.

In [ ]:
pip install ultralytics adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
   

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import SVHN
from torchvision.models import resnet18
from torch.utils.data import DataLoader

import numpy as np
from tqdm import tqdm
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
import torch.nn.functional as F
from torch.utils.data import Subset
import random

Load data:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = SVHN(root="./data", split='train', download=True, transform=transform)
test_dataset = SVHN(root="./data", split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Подготовка данных для ART
X_test = torch.stack([img for img, _ in test_dataset]).numpy()
y_test = np.array([label for _, label in test_dataset])
y_test[y_test == 10] = 0

100%|██████████| 182M/182M [00:06<00:00, 26.6MB/s]
100%|██████████| 64.3M/64.3M [00:04<00:00, 14.0MB/s]


Creating models and an ART classifier. I use Resnet18, also YOLO can be usable here too if you add an adapter described in a previous notebook - `Create_adversarial_clean.ipynb`.

In [ ]:
model = resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(0.0, 1.0),
    device_type="gpu" if torch.cuda.is_available() else "cpu"
)


Training a model with adversarial training (FGSM):

In [ ]:
adv_trainer = FastGradientMethod(estimator=classifier, eps=0.03)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"\nEpoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(train_loader, desc="Adversarial Training", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        # Сгенерировать adversarial примеры (на CPU)
        model.eval()
        adv_images = adv_trainer.generate(x=images.detach().cpu().numpy())
        adv_images = torch.tensor(adv_images).to(device).float()
        model.train()

        # Объединяем clean + adversarial
        mixed_images = torch.cat([images, adv_images])
        mixed_labels = torch.cat([labels, labels])

        optimizer.zero_grad()
        output = model(mixed_images)
        loss = criterion(output, mixed_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1} complete — Avg loss: {total_loss / len(train_loader):.4f}")



Epoch 1/5


Epoch 1 complete — Avg loss: 1.0136

Epoch 2/5


Epoch 2 complete — Avg loss: 0.6646

Epoch 3/5


Epoch 3 complete — Avg loss: 0.4617

Epoch 4/5


Epoch 4 complete — Avg loss: 0.3210

Epoch 5/5


Epoch 5 complete — Avg loss: 0.1866


Conducting an attack and checking accuracy:

In [ ]:
# 1. Переоборачиваем модель после adversarial training
classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(0.0, 1.0),
    device_type="gpu" if torch.cuda.is_available() else "cpu"
)

X_test_np = X_test.astype(np.float32)
y_test_np = y_test.astype(np.int64)

preds_clean = classifier.predict(X_test_np)
acc_clean = np.mean(np.argmax(preds_clean, axis=1) == y_test_np)
print(f"\n🎯 Accuracy on clean test set: {acc_clean * 100:.2f}%")

# 4. FGSM атака
fgsm_attack = FastGradientMethod(estimator=classifier, eps=0.03)
X_test_adv_fgsm = fgsm_attack.generate(x=X_test_np)

preds_fgsm = classifier.predict(X_test_adv_fgsm)
acc_fgsm = np.mean(np.argmax(preds_fgsm, axis=1) == y_test_np)
print(f"⚠️ Accuracy under FGSM attack: {acc_fgsm * 100:.2f}%")

# 5. PGD атака
pgd_attack = ProjectedGradientDescent(estimator=classifier, eps=0.03, eps_step=0.005, max_iter=40)
X_test_adv_pgd = pgd_attack.generate(x=X_test_np)

preds_pgd = classifier.predict(X_test_adv_pgd)
acc_pgd = np.mean(np.argmax(preds_pgd, axis=1) == y_test_np)
print(f"🔥 Accuracy under PGD attack: {acc_pgd * 100:.2f}%")



🎯 Accuracy on clean test set: 95.18%
⚠️ Accuracy under FGSM attack: 93.37%


PGD - Batches:   0%|          | 0/814 [00:00<?, ?it/s]

🔥 Accuracy under PGD attack: 6.59%


It is clear that the model is not resistant to PGD attacks. Let's work on this.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

# Функции потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Оборачиваем в ART
classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(0.0, 1.0),
)

# Настройка PGD-атаки
pgd_attack = ProjectedGradientDescent(
    estimator=classifier,
    norm=np.inf,
    eps=0.03,
    eps_step=0.005,
    max_iter=10,
    targeted=False,
    num_random_init=1,
    batch_size=32,
    verbose=False
)

# Adversarial training
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"\n🌊 Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(train_loader, desc="PGD Adversarial Training", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        # Генерация adversarial примеров (на CPU для совместимости с ART)
        model.eval()
        adv_images = pgd_attack.generate(x=images.detach().cpu().numpy())
        adv_images = torch.tensor(adv_images).to(device).float()
        model.train()

        # Объединение clean + adversarial
        mixed_images = torch.cat([images, adv_images])
        mixed_labels = torch.cat([labels, labels])

        optimizer.zero_grad()
        output = model(mixed_images)
        loss = criterion(output, mixed_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"✅ Epoch {epoch + 1} complete — Avg loss: {total_loss / len(train_loader):.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🌊 Epoch 1/5


✅ Epoch 1 complete — Avg loss: 1.1245

🌊 Epoch 2/5


✅ Epoch 2 complete — Avg loss: 0.8045

🌊 Epoch 3/5


✅ Epoch 3 complete — Avg loss: 0.7393

🌊 Epoch 4/5


✅ Epoch 4 complete — Avg loss: 0.7016

🌊 Epoch 5/5


✅ Epoch 5 complete — Avg loss: 0.6676


In [ ]:
# Преобразуем тестовый датасет в numpy (если еще не сделали)
X_test_np = np.array([x.numpy() for x, _ in test_dataset])
y_test_np = np.array([y for _, y in test_dataset])

X_test_np = X_test_np.transpose((0, 2, 3, 1))  # [N, H, W, C]
X_test_np = X_test_np.transpose((0, 3, 1, 2))  # [N, C, H, W]


In [ ]:
# FGSM
fgsm = FastGradientMethod(estimator=classifier, eps=0.03)
X_test_adv_fgsm = fgsm.generate(x=X_test_np)

# PGD
pgd = ProjectedGradientDescent(estimator=classifier, eps=0.03, eps_step=0.005, max_iter=10)
X_test_adv_pgd = pgd.generate(x=X_test_np)


PGD - Batches:   0%|          | 0/814 [00:00<?, ?it/s]

In [ ]:
# Точность на clean
pred_clean = np.argmax(classifier.predict(X_test_np), axis=1)
acc_clean = np.mean(pred_clean == y_test_np) * 100

# Точность на FGSM
pred_fgsm = np.argmax(classifier.predict(X_test_adv_fgsm), axis=1)
acc_fgsm = np.mean(pred_fgsm == y_test_np) * 100

# Точность на PGD
pred_pgd = np.argmax(classifier.predict(X_test_adv_pgd), axis=1)
acc_pgd = np.mean(pred_pgd == y_test_np) * 100

print(f"🎯 Accuracy on clean test set: {acc_clean:.2f}%")
print(f"⚠️ Accuracy under FGSM attack: {acc_fgsm:.2f}%")
print(f"🔥 Accuracy under PGD attack: {acc_pgd:.2f}%")


🎯 Accuracy on clean test set: 92.85%
⚠️ Accuracy under FGSM attack: 63.01%
🔥 Accuracy under PGD attack: 54.61%


Great, after training, the model became much more resistant to PGD attacks, but the quality of FGSM attacks decreased. This means that we need to train on a dataset that includes images of both attacks if we want to protect ourselves from them.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

svhn_train = SVHN(root="./data", split='train', download=True, transform=transform)

# Преобразуем SVHN в тензоры
X_train_tensor = torch.stack([img for img, _ in svhn_train])
y_train_tensor = torch.tensor([label if label != 10 else 0 for _, label in svhn_train])  # исправляем метки

y_train_tensor = y_train_tensor.long()
y_train_tensor = y_train_tensor.long()



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
])

class AugmentedTensorDataset(torch.utils.data.Dataset):
    def __init__(self, tensors, transform=None):
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.tensors[0][index], self.tensors[1][index]
        img = transforms.ToPILImage()(x)
        if self.transform:
            img = self.transform(img)
        x = transforms.ToTensor()(img)
        return x, y

    def __len__(self):
        return len(self.tensors[0])

indices = random.sample(range(len(train_dataset)), 10000)  # Например, 10k примеров вместо 73k
train_dataset = Subset(train_dataset, indices)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
class ResNetWithDropout(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super().__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(base_model.fc.in_features, 10)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

base_model = resnet18(weights=None)
base_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
base_model.maxpool = nn.Identity()
model = ResNetWithDropout(base_model).to(device)

In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, pred, target):
        log_probs = F.log_softmax(pred, dim=-1)
        true_dist = torch.zeros_like(log_probs)
        true_dist.fill_(self.smoothing / (pred.size(1) - 1))
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    clip_values=(0.0, 1.0),
    device_type='gpu' if torch.cuda.is_available() else 'cpu'
)

fgsm = FastGradientMethod(estimator=classifier, eps=0.03)
pgd = ProjectedGradientDescent(estimator=classifier, eps=0.03, max_iter=10)

# Обучение на смеси атак ===
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"\n🌪 Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(train_loader, desc="Mixed Adversarial Training")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        model.eval()
        fgsm_adv = fgsm.generate(x=images.detach().cpu().numpy())
        pgd_adv = pgd.generate(x=images.detach().cpu().numpy())

        fgsm_adv = torch.tensor(fgsm_adv).to(device).float()
        pgd_adv = torch.tensor(pgd_adv).to(device).float()
        model.train()

        # Объединяем clean + FGSM + PGD
        mixed_images = torch.cat([images, fgsm_adv, pgd_adv])
        mixed_labels = torch.cat([labels, labels, labels])

        optimizer.zero_grad()
        outputs = model(mixed_images)
        loss = criterion(outputs, mixed_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    print(f"✅ Epoch {epoch + 1} complete — Avg loss: {total_loss / len(train_loader):.4f}")


🌪 Epoch 1/5


Mixed Adversarial Training:   0%|          | 0/79 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Mixed Adversarial Training:   1%|▏         | 1/79 [00:01<02:34,  1.98s/it, loss=2.48]

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Mixed Adversarial Training:  99%|█████████▊| 78/79 [02:30<00:01,  1.92s/it, loss=1.22]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Mixed Adversarial Training: 100%|██████████| 79/79 [02:31<00:00,  1.91s/it, loss=1.53]

✅ Epoch 5 complete — Avg loss: 1.2303


Accuracy evaluation on clean + FGSM + PGD:

In [ ]:
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import torch.nn.functional as F

# Убедимся, что модель в режиме инференса
model.eval()

# 1. Точность на чистых данных
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

acc_clean = 100.0 * correct / total
print(f"🎯 Accuracy on clean test set: {acc_clean:.2f}%")

# 2. Генерация adversarial атак (по батчам)
def evaluate_adversarial(attack):
    correct = 0
    total = 0
    model.eval()

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Переводим в numpy и атакуем
        adv_images = attack.generate(x=images.detach().cpu().numpy())

        # Переводим обратно в тензор
        adv_images = torch.tensor(adv_images).to(device).float()

        with torch.no_grad():
            outputs = model(adv_images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return 100.0 * correct / total

# 3. FGSM
fgsm = FastGradientMethod(estimator=classifier, eps=0.03)
acc_fgsm = evaluate_adversarial(fgsm)
print(f"⚠️ Accuracy under FGSM attack: {acc_fgsm:.2f}%")

# 4. PGD
pgd = ProjectedGradientDescent(estimator=classifier, eps=0.03, eps_step=0.007, max_iter=40)
acc_pgd = evaluate_adversarial(pgd)
print(f"🔥 Accuracy under PGD attack: {acc_pgd:.2f}%")


🎯 Accuracy on clean test set: 75.06%
⚠️ Accuracy under FGSM attack: 41.10%


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]

🔥 Accuracy under PGD attack: 31.80%


From the results, we can see that the accuracy has decreased on both normal and attacked images. This may be due to the fact that I trained on a subset of 79 values (instead of the full dataset as in the previous training), with a batch size of 256 (instead of 128). However, training would have taken approximately 5 hours if I had used the full dataset.

It is also worth considering the following factors:

During adversarial training, the model learns to be more resistant to attacks, but at the same time, it loses some of its accuracy on regular data.
In other words, improving resistance often comes at the cost of reducing pure accuracy.

**The complexity of mixed adversarial training**
Training on two types of attacks (FGSM and PGD) simultaneously is more challenging and may require more careful selection of hyperparameters (learning rate, epochs, batch size). Otherwise, the model may get confused or retrain on simpler examples.

The task does not specify whether the model should be able to adapt to one or multiple attacks. Therefore, in this case, the resnet18 model, after being trained on a modified dataset, became more resistant to the PGD attack (with an accuracy that increased from 6.59% to 54.61%).